# buy-and-hold

buy, then never ever sell, until the end date :)

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# fetch timeseries, select, finalize
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False, pad=False)
ts['adj_close'] = ts['close']
ts, start = pf.finalize_timeseries(ts, start)

# create tradelog and daily balance objects
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,43.97,43.75,43.97,43.94,1003200.00,43.94
1993-02-01,44.25,43.97,43.97,44.25,480500.00,44.25
1993-02-02,44.38,44.12,44.22,44.34,201300.00,44.34
1993-02-03,44.84,44.38,44.41,44.81,529400.00,44.81
1993-02-04,45.09,44.47,44.97,45.00,531500.00,45.00
...,...,...,...,...,...,...
2020-07-08,316.30,312.70,314.61,316.18,54638600.00,316.18
2020-07-09,317.10,310.68,316.84,314.38,83354200.00,314.38
2020-07-10,317.88,312.76,314.31,317.59,57550400.00,317.59


Algorithm

In [5]:
pf.TradeLog.cash = capital

# loop through timeseries
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # buy
    if tlog.shares == 0:
        tlog.buy(date, row.close)
    # sell
    elif end_flag:
        tlog.sell(date, row.close)

    # record daily balance
    dbal.append(date, row.high, row.low, row.close)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
0,1993-01-29,43.94,2020-07-14,316.03,272.09,61765.00,227,61765.00,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,7.44
max_closed_out_drawdown,-56.43
best_month,23.06
worst_month,-31.38
sharpe_ratio,0.47
sortino_ratio,0.60
monthly_std,4.49
